In [1]:
# install.packages("rvest")
# remotes::install_github("dmi3kno/polite")("politer")
# install.packages("xml2")
library(rvest) # rvest makes scraping easier
library(polite) # polite is the "polite" version of rvest
library(xml2) # makes it easier to work with HTML and XML from R


library(tidyverse)
library(magrittr)
library(purrr) 
library(glue) 
library(stringr) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()         masks stats::filter()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()

Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract




## Use API

To get the weather history data we use api from this website https://www.visualcrossing.com/ . The api from that website allow us to get various kind of information about the weather in specific city and in certain range of time. However in our case we only interested about weather history from a week ago to the current date.

In [5]:
#first we load the httr library so that we can fetch the response from an api
library(httr)

In [19]:
# this is helper function to get the actually desired data inside the list
# this function takes 2 parameter, the first one is the data which is a list of all the information about the weather for each day, the second parameter is the index which is a string of the data we want to get
get_information_list <- function(data, index) {
    empty <- vector(mode = "list", length = 0) # first we prepare an empty list which will be appended with the information we want to get
    for(i in seq_along(data)){ # next we loop through every item inside the data given
        empty <- append(empty, data[i][[1]][[index]]) # then we append the data to the list we prepared before
    }
    return(empty)
}

#this is the function to get the weather history from a week ago until the current date
#this function will return a tibble which contains the information about the weather from 7 days ago to the current date. The information includes
get_weather_data <- function(city){

    currentDate <- Sys.Date()
    
    lastWeek <- as.Date(currentDate) - 7
    

    response_raw <- GET(glue("https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/{lastWeek}/{currentDate}?unitGroup=metric&key=GMTLW3T9CX22FK8GH662Q9GPZ&contentType=json")) 
    # response_raw
    response <- content(response_raw) # here I get the body content of the response


    result_weather_data <- tibble(Date = get_information_list(response[["days"]], "datetime"),
                                    Temperature = get_information_list(response[["days"]], "temp"),
                                    WindSpeed = get_information_list(response[["days"]], "windspeed"),
                                    Humidity = get_information_list(response[["days"]], "humidity"),
                                    Conditions = get_information_list(response[["days"]], "conditions"))
    result_weather_data <- result_weather_data %>% mutate(across(Date, as.character)) %>%
                                                    mutate(across(Temperature, as.double)) %>%
                                                    mutate(across(WindSpeed, as.double)) %>%
                                                    mutate(across(Humidity, as.double)) %>%
                                                    mutate(across(Conditions, as.character))
    return(result_weather_data)
}

the_data<-get_weather_data("christchurch")
the_data

ERROR: Error: lexical error: invalid char in json text.
                                       You have exceeded the maximum n
                     (right here) ------^

